### SQL in Python Excercise

In [3]:
# <!-- 
# - 
#     - Create a notebook which does following:
#         - Insert at least **3 different coins** into the `coins` table.
#         - Insert **5 daily price records** for each coin into the `prices` table.
#         - Create a business question (what is the average price for each coin? what is the highest price for each coin?)
#         - Write an SQL query to solve the business question -->

### Create new Database and Tables

In [4]:
import sqlite3
# create a new database
with sqlite3.connect("crypto2.db") as conn:
    cursor = conn.cursor()
# create a new table named coins
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS coins (
        Coin_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        symbol TEXT NOT NULL UNIQUE
    )
    """)

    # create a new table named prices
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS prices (
        Price_id INTEGER PRIMARY KEY AUTOINCREMENT,
        coin_id INTEGER,
        price REAL,
        date DATE,
        FOREIGN KEY (coin_id) REFERENCES coins (Coin_id)
    )
    """)

### Insert at least **3 different coins** into the `coins` table.

In [5]:
# Insert at least **3 different coins** into the `coins` table.
data_coins = [
    ('Bitcoin', 'BTC'),
    ('Ethereum', 'ETH'),
    ('Ripple', 'XRP')
]
with sqlite3.connect("crypto2.db") as conn:
    cursor = conn.cursor()

# Insert coins into the coins table
    cursor.executemany("INSERT OR IGNORE INTO coins (name, symbol) VALUES (?, ?)", data_coins)

# select all coins
    cursor.execute("SELECT * FROM coins")
    coins = cursor.fetchall()

# convert to pandas dataframe
import pandas as pd
df_coins = pd.DataFrame(coins, columns=['Coin_id', 'Name', 'Symbol'])
print(df_coins)

   Coin_id      Name Symbol
0        1   Bitcoin    BTC
1        2  Ethereum    ETH
2        3    Ripple    XRP


### Insert **5 daily price records** for each coin into the `prices` table.

In [6]:
# Insert **5 daily price records** for each coin into the `prices` table.
data_prices = [
    (1, 50000.0, '2023-10-01'),
    (1, 51000.0, '2023-10-02'),
    (1, 52000.0, '2023-10-03'),
    (1, 53000.0, '2023-10-04'),
    (1, 54000.0, '2023-10-05'),
    (2, 3000.0, '2023-10-01'),
    (2, 3100.0, '2023-10-02'),
    (2, 3200.0, '2023-10-03'),
    (2, 3300.0, '2023-10-04'),
    (2, 3400.0, '2023-10-05'),
    (3, 1.0, '2023-10-01'),
    (3, 1.1, '2023-10-02'),
    (3, 1.2, '2023-10-03'),
    (3, 1.3, '2023-10-04'),
    (3, 1.4, '2023-10-05')
]
with sqlite3.connect("crypto2.db") as conn:
    cursor = conn.cursor()
    cursor.executemany("INSERT INTO prices (coin_id, price, date) VALUES (?, ?, ?)", data_prices)

# select all prices
    cursor.execute("SELECT * FROM prices")
    prices = cursor.fetchall()
   
# convert to pandas dataframe
df_prices = pd.DataFrame(prices, columns=['Price_id', 'Coin_id', 'Price', 'Date'])
print(df_prices.head())


   Price_id  Coin_id    Price        Date
0         1        1  50000.0  2023-10-01
1         2        1  51000.0  2023-10-02
2         3        1  52000.0  2023-10-03
3         4        1  53000.0  2023-10-04
4         5        1  54000.0  2023-10-05


### Join the coins and prices tables

In [ ]:
# Join the coins and prices tables

merged_df = pd.merge(df_coins, df_prices, on='Coin_id')

# Business Question: What is the average price for each coin?

cursor.execute("""
SELECT c.name, AVG(p.price) as average_price
FROM coins c
JOIN prices p ON c.coin_id = p.coin_id
GROUP BY c.name
""")

average_prices = cursor.fetchall()
# convert to pandas dataframe
df_average = pd.DataFrame(average_prices, columns=['Coin', 'Average Price'])
print(df_average)

       Coin  Average Price
0   Bitcoin        52000.0
1  Ethereum         3200.0
2    Ripple            1.2


### What is the highest price for each coin

In [12]:
# what is the highest price for each coin

cursor.execute("""
SELECT c.name, MAX(p.price) as highest_price
FROM coins c
JOIN prices p ON c.coin_id = p.coin_id
GROUP BY c.name
""")

highest_prices = cursor.fetchall()
# convert to pandas dataframe
df_highest = pd.DataFrame(highest_prices, columns=['Coin', 'Highest Price'])
print(df_highest)

       Coin  Highest Price
0   Bitcoin        54000.0
1  Ethereum         3400.0
2    Ripple            1.4
